In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import csv
import random
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [5]:
def get_result(code,page):
    headers = { "User-Agent": f"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.{random.randrange(99)} (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36"}
    baseurl = 'https://www.realtor.com/soldhomeprices/{0}/type-single-family-home,condo-townhome-row-home-co-op/pg-{1}'.format(code,page)
    r = requests.get(baseurl,headers = headers)
    return r.content

In [6]:
def get_first_page(code):
    headers = { "User-Agent": f"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.{random.randrange(99)} (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36"}
    baseurl = 'https://www.realtor.com/soldhomeprices/{0}/type-single-family-home,condo-townhome-row-home-co-op'.format(code)
    r = requests.get(baseurl,headers = headers)
    return r.content

In [7]:
def zipcode_list(path,colname):
    return pd.read_excel(path)[colname]

In [8]:
def get_house_page(url):
    headers = { "User-Agent": f"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.{random.randrange(99)} (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36"}
    #cookie = {"RUL=EPKxyP4FGPKYzY0GIrMBATZhkOPkdsbrqcjiflWTIOIuIH6pPkf4YTCL7xLOnFA-eTRtbebms-UFE_BkgqK3ONkDtMTFbWRLRKXFRW3Q22x7dYRxHHbu_-gsZ1eH558qMu3TR_0XgatrTEq8S7E0WN1TKsarRrxwkboyNBbq7Pao7AdpFluylPZOuc5MABlDBk053qZahOm0YySGedoxtabX7wBeQRMnrfPEQ8EiGyGxHwUJxg5HKkaW2ynrzjLtwq0|cs=AP6Md-Wdq_czgxUddMCJuXxJbMiT; IDE=AHWqTUkpnaoHwOW1Z7vjzclkzBEX4CXUaz8ABc0Xcjw2JDTTkS3xfoLbXbjiJT-6ryA"}
    return requests.get('https://www.realtor.com'+url,headers= headers).content

In [9]:
def get_b_count(soup):
    beds =soup.find_all(attrs={"data-label": "property-meta-beds"})[0].contents[1].string
    baths = soup.find_all(attrs={"data-label": "property-meta-bath"})[0].contents[1].string
    sqft = soup.find_all(attrs={"data-label": "property-meta-sqft"})[0].contents[1].string
    return (beds,baths,sqft)

In [10]:
def get_price(soup):
    return soup.find_all(attrs={"class": "display-inline"})[2].find_all('span')[1].string.strip()

In [11]:
def get_type_build(soup):
    temp = soup.find_all(attrs={'class':'key-fact-data ellipsis'})
    return (temp[2].string,temp[3].string)

In [12]:
def get_neigh_page(soup):
    return 'https://www.realtor.com'+soup.find_all(attrs={'class':'neighborhood-max-width-sm padding-bottom'})[0].a['href']

In [13]:
def get_school(page):
    result=dict()
    try:
        for school in page.find_all(attrs={'class':"table table-clear table-heading-unstyled table-school school-rating-lg"})[0].tbody.find_all('tr'):
            result[school.a.string]= dict()
            result[school.a.string]['rating'] = school.span.string
            result[school.a.string]['grades'] = school.find_all('td',attrs={'class':True})[1].string
            result[school.a.string]['distance'] = school.find_all('td',attrs={'class':True})[2].string
    except IndexError:
        return ''
        
    return result

In [14]:
def create_csv(p, head: list):
    path = p
    with open(path, "w") as f:
        csv_write = csv.writer(f)
        csv_head = head
        csv_write.writerow(csv_head)
def write_csv(p, data: list):
    with open(p, "a+") as f:
        csv_write = csv.writer(f)
        csv_write.writerow(data)

In [15]:
def get_house_list(code:int):
    house_list = set()
    page = 1
    soup = BeautifulSoup(get_result(code,1))
    maxpage = int(soup.find_all(attrs = {'data-omtag':"not_for_sale:srp_list:paging:4"})[0].string)
    while(page <= maxpage):
        soup = BeautifulSoup(get_result(code,page))
        for i in soup.find_all('a'):
            try:
                if 'realestateandhomes-detail' in i['href']:
                    house_list.add(i['href'])
            except KeyError:
                pass
            page +=1
    return house_list

In [50]:
def get_house_link(soup):
    temp = set()
    for i in soup.find_all('a'):
        try:
            if "/realestateandhomes-detail" in i['href']:
                temp.add(i['href'])
        except:
            KeyError
    return list(temp)

In [85]:
def generate_detail(house_list:list,p):
    head = ['ID',"beds","baths","sqft","price",'type',"built","neigborhood","school"]
    create_csv(p,head)
    ##driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')
    for link in house_list:
        print(link)
        name = link.split('/')[-1]
        ##driver.get(get_house_page(link))
        ##time.sleep(3)
        temp =get_house_page(link)
        soup = BeautifulSoup(temp)
        beds,baths,sqft =get_b_count(soup)
        price = get_price(soup)
        t,built = get_type_build(soup)
        nei = get_neigh_page(soup)
        school = get_school(soup)
        write_csv(p,[name,beds,baths,sqft,price,t,built,nei,school])
        sleep_t = random.randrange(5,30)
        time.sleep(sleep_t)

In [1]:
input_list = total
generate_detail(input_list,'text2.csv')

In [76]:
t = get_house_link(soup)

In [77]:
t

['/realestateandhomes-detail/3371-Erinlea-Ave_Thousand-Oaks_CA_91320_M14310-99766',
 '/realestateandhomes-detail/25-Cedar-Crest-Ct_Thousand-Oaks_CA_91320_M20471-43062',
 '/realestateandhomes-detail/3832-Calle-Linda-Vista_Newbury-Park_CA_91320_M17695-83048',
 '/realestateandhomes-detail/167-Brook-Rd_Thousand-Oaks_CA_91320_M17361-58771',
 '/realestateandhomes-detail/4043-Calle-Mira-Monte_Thousand-Oaks_CA_91320_M10579-67256',
 '/realestateandhomes-detail/1217-Knollwood-Dr_Thousand-Oaks_CA_91320_M14035-22969',
 '/realestateandhomes-detail/1253-Rotella-St_Thousand-Oaks_CA_91320_M14075-10424',
 '/realestateandhomes-detail/802-Deer-Willow-Ct_Thousand-Oaks_CA_91320_M14327-39467',
 '/realestateandhomes-detail/33-Greenmeadow-Dr_Thousand-Oaks_CA_91320_M24066-34217',
 '/realestateandhomes-detail/1772-Blue-Canyon-St_Thousand-Oaks_CA_91320_M15927-59425',
 '/realestateandhomes-detail/8-Bluefield-Ave_Thousand-Oaks_CA_91320_M14829-36692',
 '/realestateandhomes-detail/1562-Silver-Shadow-Dr_Newbury-Park_

In [81]:
total

['/realestateandhomes-detail/3526-Robinwood-Ave_Clovis_CA_93619_M24662-66816',
 '/realestateandhomes-detail/2881-Huntington-Blvd-Apt-255_Fresno_CA_93721_M28100-58589',
 '/realestateandhomes-detail/1042-Ezie-Ave_Clovis_CA_93611_M21328-41920',
 '/realestateandhomes-detail/7319-N-Lola-Ave_Fresno_CA_93722_M23004-26864',
 '/realestateandhomes-detail/2811-E-Fedora-Ave_Fresno_CA_93726_M22041-43041',
 '/realestateandhomes-detail/14243-S-Marquette-Ave_Chicago_IL_60633_M75072-83519',
 '/realestateandhomes-detail/12853-S-Escanaba-Ave_Chicago_IL_60633_M77278-48038',
 '/realestateandhomes-detail/12634-S-Muskegon-Ave_Chicago_IL_60633_M80141-13636',
 '/realestateandhomes-detail/1741-Robinwood-Ave_Clovis_CA_93611_M20898-74288',
 '/realestateandhomes-detail/3769-Chessa-Ln_Clovis_CA_93619_M25990-70622',
 '/realestateandhomes-detail/5175-Via-El-Molino_Newbury-Park_CA_91320_M20729-75695',
 '/realestateandhomes-detail/3094-W-Mesa-Ave_Fresno_CA_93711_M26566-74752',
 '/realestateandhomes-detail/204-S-J-St_Ma

In [80]:
random.shuffle(total)

In [67]:
house_list = []
for i in soup.find_all('a'):
    try:
        if 'realestateandhomes-detail' in i['href']:
            house_list.append(i['href'])
    except KeyError:
        pass

In [281]:
with open('link_list.txt', 'w') as filehandle:
    for listitem in house_list:
        filehandle.write('%s\n' % listitem)